In [3]:
import pandas as pd
import duckdb
import requests
import geopy as gp
from geopy.distance import geodesic as GD
import geopandas as gpd 
from shapely.geometry import Point
import argparse
import sys


In [4]:
from shapely.geometry import Point
import geopandas as gpd   # conda install -c conda-forge geopandas

def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

In [5]:
#STEP 1: Connecting to the database and extracting data from BICIMAD
con = duckdb.connect('./Data/bicimad.db') #connecting to the Database(db)file.
con
##usado en ACQUISITION

In [6]:
#To extract information, it´s neccesary to connect to SQL(con.sql)
#At the same time, convert it to DF.
bicimad_df= con.sql("SELECT * FROM bicimad_stations").df()
##usado en ACQUISITION

In [7]:
bicimad_df
##usado en ACQUISITION

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates
0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]"
1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]"
2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]"
3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]"
4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,265,257 - INEF,0,257,Avenida Juan de Herrera frente a la calle Paul...,1,0,24,3,19,0,Point,"[-3.72997, 40.43896]"
260,266,258 - Ciudad Universitaria 1,2,258,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,9,14,0,Point,"[-3.72699, 40.44375]"
261,267,259 - Ciudad Universitaria 2,2,259,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,12,10,0,Point,"[-3.72693, 40.44342]"
262,268,260 - Facultad Biología,2,260,Calle José Antonio Novais frente al nº 12,1,0,24,12,10,0,Point,"[-3.7272945, 40.4483322]"


In [8]:
bicimad_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    264 non-null    int64 
 1   name                  264 non-null    object
 2   light                 264 non-null    int64 
 3   number                264 non-null    object
 4   address               264 non-null    object
 5   activate              264 non-null    int64 
 6   no_available          264 non-null    int64 
 7   total_bases           264 non-null    int64 
 8   dock_bikes            264 non-null    int64 
 9   free_bases            264 non-null    int64 
 10  reservations_count    264 non-null    int64 
 11  geometry.type         264 non-null    object
 12  geometry.coordinates  264 non-null    object
dtypes: int64(8), object(5)
memory usage: 26.9+ KB


In [9]:
bicimad_df.columns

Index(['id', 'name', 'light', 'number', 'address', 'activate', 'no_available',
       'total_bases', 'dock_bikes', 'free_bases', 'reservations_count',
       'geometry.type', 'geometry.coordinates'],
      dtype='object')

In [10]:
bicimad_df_filter= bicimad_df[['name', 'address', 'geometry.coordinates']]
bicimad_df_filter

,name,address,geometry.coordinates
0,1a - Puerta del Sol A,Puerta del Sol nº 1,"[-3.7018341, 40.4172137]"
1,1b - Puerta del Sol B,Puerta del Sol nº 1,"[-3.701602938060457, 40.41731271011562]"
2,2 - Miguel Moya,Calle Miguel Moya nº 1,"[-3.7058415, 40.4205886]"
3,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,"[-3.7069171, 40.4302937]"
4,4 - Malasaña,Calle Manuela Malasaña nº 5,"[-3.7025875, 40.4285524]"
...,...,...,...
259,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,"[-3.72997, 40.43896]"
260,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,"[-3.72699, 40.44375]"
261,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,"[-3.72693, 40.44342]"
262,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,"[-3.7272945, 40.4483322]"


In [11]:
lat_lon= bicimad_df_filter["geometry.coordinates"].str.strip('[]').str.split(",", expand=True)

In [12]:
bicimad_df_filter

,name,address,geometry.coordinates
0,1a - Puerta del Sol A,Puerta del Sol nº 1,"[-3.7018341, 40.4172137]"
1,1b - Puerta del Sol B,Puerta del Sol nº 1,"[-3.701602938060457, 40.41731271011562]"
2,2 - Miguel Moya,Calle Miguel Moya nº 1,"[-3.7058415, 40.4205886]"
3,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,"[-3.7069171, 40.4302937]"
4,4 - Malasaña,Calle Manuela Malasaña nº 5,"[-3.7025875, 40.4285524]"
...,...,...,...
259,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,"[-3.72997, 40.43896]"
260,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,"[-3.72699, 40.44375]"
261,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,"[-3.72693, 40.44342]"
262,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,"[-3.7272945, 40.4483322]"


In [13]:
bicimad_df_filter = bicimad_df_filter.assign(latitud=lat_lon[1].astype(float), longitud=lat_lon[0].astype(float))
#bicimad_df_filter = pd.concat([bicimad_df_filter, lat_lon], axis=1) #Concat el nuevo DF con el anterior
#bicimad_df_filter = pd.concat([bicimad_df_filter, lat_lon], axis=1) #Concat el nuevo DF con el anterior
#dos últimas filas codigo largo

In [14]:
#bicimad_df_filter['latitud'] = bicimad_df_filter['latitud'].astype('float')  ##con el método astype consigo cambiar los datosa de la columna latitud de string a float
# No me ha dejado hacer las dos columnas a la vez.
#bicimad_df_filter = bicimad_df_filter.astype({'latitud':'float', 'longitud':'float'}) #latitud to float
#bicimad_df_filter['longitud'] = bicimad_df_filter['longitud'].astype('float') #longitud to float
#bicimad_df_filter = pd.concat([bicimad_df_filter, lat_lon], axis=1) #Concat el nuevo DF con el anterior

In [15]:
bicimad_df_filter

,name,address,geometry.coordinates,latitud,longitud
0,1a - Puerta del Sol A,Puerta del Sol nº 1,"[-3.7018341, 40.4172137]",40.417214,-3.701834
1,1b - Puerta del Sol B,Puerta del Sol nº 1,"[-3.701602938060457, 40.41731271011562]",40.417313,-3.701603
2,2 - Miguel Moya,Calle Miguel Moya nº 1,"[-3.7058415, 40.4205886]",40.420589,-3.705842
3,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,"[-3.7069171, 40.4302937]",40.430294,-3.706917
4,4 - Malasaña,Calle Manuela Malasaña nº 5,"[-3.7025875, 40.4285524]",40.428552,-3.702587
...,...,...,...,...,...
259,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,"[-3.72997, 40.43896]",40.438960,-3.729970
260,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,"[-3.72699, 40.44375]",40.443750,-3.726990
261,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,"[-3.72693, 40.44342]",40.443420,-3.726930
262,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,"[-3.7272945, 40.4483322]",40.448332,-3.727295


In [16]:
bicimad_df_final= bicimad_df_filter.rename(columns={'name':'nameBM','address':'addressBM', 'latitud':'latitudeBM', 'longitud':'longitudeBM'}).drop(['geometry.coordinates'], axis=1)
#simplifica
#bicimad_df_final= bicimad_df_filter.rename(columns={'name':'nameBM','address':'addressBM', 'latitud':'latitudeBM', 'longitud':'longitudeBM'}).drop(['geometry.coordinates'], axis=1)
#bicimad_df_final=bicimad_df_filter[['name','address', 'latitud', 'longitud']]

In [17]:
bicimad_df_final

,nameBM,addressBM,latitudeBM,longitudeBM
0,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834
1,1b - Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603
2,2 - Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842
3,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917
4,4 - Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587
...,...,...,...,...
259,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,40.438960,-3.729970
260,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,40.443750,-3.726990
261,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,40.443420,-3.726930
262,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,40.448332,-3.727295


In [18]:
bicimad_df_final['pointBM'] = bicimad_df_final.apply(lambda x:(gpd.GeoSeries([Point(x['latitudeBM'], x['longitudeBM'])], crs=4326).to_crs(3857)), axis=1)


In [19]:
bicimad_df_final

,nameBM,addressBM,latitudeBM,longitudeBM,pointBM
0,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,POINT (4499223.648 -412373.285)
1,1b - Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603,POINT (4499234.670 -412347.499)
2,2 - Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842,POINT (4499599.341 -412820.321)
3,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917,POINT (4500679.707 -412940.307)
4,4 - Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587,POINT (4500485.867 -412457.329)
...,...,...,...,...,...
259,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,40.438960,-3.729970,POINT (4501644.435 -415511.958)
260,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,40.443750,-3.726990,POINT (4502177.656 -415179.522)
261,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,40.443420,-3.726930,POINT (4502140.920 -415172.829)
262,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,40.448332,-3.727295,POINT (4502687.744 -415213.491)


In [20]:
type(bicimad_df_final)

pandas.core.frame.DataFrame

In [21]:
bicimad_df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   nameBM       264 non-null    object  
 1   addressBM    264 non-null    object  
 2   latitudeBM   264 non-null    float64 
 3   longitudeBM  264 non-null    float64 
 4   pointBM      264 non-null    geometry
dtypes: float64(2), geometry(1), object(2)
memory usage: 10.4+ KB


In [22]:
#STEP 2: Connecting to the REST API and extracting information.
#API data request
#Getting the content of the request/response in json format (Python object). It is easier to handle.
c_cult_json_data = requests.get('https://datos.madrid.es/egob/catalogo/200304-0-centros-culturales.json').json()
##usado en ACQUISITION

In [23]:
print(type(c_cult_json_data))

<class 'dict'>


In [24]:
#c_cult_json_data = response.json()
#Getting the content of the request/response in json format (Python object). It is easier to handle.

In [25]:
type(c_cult_json_data)
##usado en ACQUISITION


dict

In [26]:
c_cult_json_data.keys()

dict_keys(['@context', '@graph'])

In [27]:
c_cult_pd1 = pd.DataFrame(c_cult_json_data['@context'], index=[0])


In [28]:
c_cult_json_data['@context']
##usado en ACQUISITION, aunque no es el que cogemos

{'c': 'http://www.w3.org/2002/12/cal#',
 'dcterms': 'http://purl.org/dc/terms/',
 'geo': 'http://www.w3.org/2003/01/geo/wgs84_pos#',
 'loc': 'http://purl.org/ctic/infraestructuras/localizacion#',
 'org': 'http://purl.org/ctic/infraestructuras/organizacion#',
 'vcard': 'http://www.w3.org/2006/vcard/ns#',
 'schema': 'https://schema.org/',
 'title': 'vcard:fn',
 'id': 'dcterms:identifier',
 'relation': 'dcterms:relation',
 'references': 'dcterms:references',
 'address': 'vcard:adr',
 'area': 'loc:barrio',
 'district': 'loc:distrito',
 'locality': 'vcard:locality',
 'postal-code': 'vcard:postal-code',
 'street-address': 'vcard:street-address',
 'location': 'vcard:geo',
 'latitude': 'geo:lat',
 'longitude': 'geo:long',
 'organization': 'vcard:org',
 'organization-desc': 'dcterms:description',
 'accesibility': 'org:accesibilidad',
 'services': 'org:servicios',
 'schedule': 'org:horario',
 'organization-name': 'vcard:organization-name',
 'description': 'c:summary',
 'link': 'c:url',
 'uid': '

In [29]:
c_cult_json_data['@graph']
##usado en ACQUISITION, es el que cogemos

[{'@id': 'https://datos.madrid.es/egob/catalogo/tipo/entidadesyorganismos/5989871-auditorio-carmen-laforet-ciudad-lineal-.json',
  '@type': 'https://datos.madrid.es/egob/kos/entidadesYorganismos/AuditoriosSalasConcierto',
  'id': '5989871',
  'title': 'Auditorio Carmen Laforet (Ciudad Lineal)',
  'relation': 'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=bfa48ab43d6bb410VgnVCM100000171f5a0aRCRD&vgnextoid=fca22248514ae210VgnVCM1000000b205a0aRCRD',
  'address': {'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/CiudadLineal'},
   'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/CiudadLineal/Barrio/Costillares'},
   'locality': 'MADRID',
   'postal-code': '28033',
   'street-address': 'CALLE JAZMIN 46'},
  'location': {'latitude': 40.47960826297022, 'longitude': -3.665799020694628},
  'organization': {'organization-desc': 'Ubicado en el Centro de Mayores Carmen Laforet\xa0\xa0Metro: Pinar de 

In [30]:
print(type(c_cult_json_data))
#json returns a dictionary data type
##usado en ACQUISITION

<class 'dict'>


In [31]:
print(len(c_cult_json_data))

2


In [32]:
#c_cult_pd1 = pd.DataFrame(c_cult_json_data['@context'], index=[0])
#print(c_cult_pd)
#Not possible
#ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.
##NO SE NECESITA

In [33]:
#c_cult_pd1

In [34]:
#c_cult_pd1.info()

In [35]:
#c_cult_pd2 = pd.DataFrame(c_cult_json_data['@graph'])
#c_cult_pd2 

In [36]:
#c_cult_pd2.info()

In [37]:
#c_cult_pd2.columns

In [38]:
c_cult_pd2= pd.json_normalize(c_cult_json_data['@graph']) 
#con esta función el json se convierte en una tabla plana
##usado en ACQUISITION

In [39]:
c_cult_pd2

,@id,@type,id,title,relation,address.district.@id,address.area.@id,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,organization.organization-desc,organization.accesibility,organization.schedule,organization.services,organization.organization-name
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,5989871,Auditorio Carmen Laforet (Ciudad Lineal),http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28033,CALLE JAZMIN 46,40.479608,-3.665799,Ubicado en el Centro de Mayores Carmen Laforet...,1,De 9 a 21 horas.,Aforo: 190 personas Zona wifi Cafetería,Auditorio Carmen Laforet (Ciudad Lineal)
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,2389,Auditorio al aire libre. Parque del Calero,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28027,CALLE JOSE DEL HIERRO 3,40.436300,-3.650410,Mantenimiento: Dependiente de la Junta Munici...,1,,,Auditorio al aire libre. Parque del Calero
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,139451,Auditorio y sala de exposiciones Paco de Lucía...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28044,AVENIDA LAS AGUILAS 2,40.384939,-3.764196,Metro de Madrid. Línea 5. Estación: Aluche. ...,1,,Sala de exposiciones. Salón de actos y auditor...,Auditorio y sala de exposiciones Paco de Lucía...
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,84763,Centro Comunitario Casino de la Reina,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28005,CALLE CASINO 3,40.406818,-3.704690,"Metro: Embajadores (líneas 3 y 5) Bus: M1, ...",1,"De apertura De lunes a viernes, de 8:30 a 21...",Orientación sobre recursos sociales a vecinos....,Centro Comunitario Casino de la Reina
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,1914,Centro Cultural - Centro Socio Cultural Villa ...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28042,CALLE BOTICA 7 y 10,40.473402,-3.578523,"Metro: Barajas (línea 8) Bus: 101, 105, 115",1,"De lunes a viernes, de 9 a 14 y de 15 a 17 hor...",3 salas de exposiciones Salón de actos / audit...,Centro Cultural - Centro Socio Cultural Villa ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,4776440,Medialab en Matadero. Matadero Madrid,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28045,PASEO CHOPERA 14,40.392458,-3.697207,Medialab Matadero es un programa del Área de G...,1,,,Medialab en Matadero. Matadero Madrid
114,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,10189992,Nave de Terneras del Centro Cultural Casa del ...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28045,PASEO CHOPERA 10,40.393433,-3.698578,"Metro: Legazpi (líneas 3, 6) Bus: 62, 6, 78...",1,Agosto cerrado.,Wifi Aforo: 492 personas,Nave de Terneras del Centro Cultural Casa del ...
115,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,11956343,Sala de exp

In [40]:
type(c_cult_pd2)

pandas.core.frame.DataFrame

In [41]:
c_cult_pd2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   @id                             118 non-null    object 
 1   @type                           118 non-null    object 
 2   id                              118 non-null    object 
 3   title                           118 non-null    object 
 4   relation                        118 non-null    object 
 5   address.district.@id            118 non-null    object 
 6   address.area.@id                118 non-null    object 
 7   address.locality                118 non-null    object 
 8   address.postal-code             118 non-null    object 
 9   address.street-address          118 non-null    object 
 10  location.latitude               118 non-null    float64
 11  location.longitude              118 non-null    float64
 12  organization.organization-desc  118 

In [42]:
#c_cult_pd2_final= c_cult_pd2[['@type','title','address.street-address', 'location.latitude', 'location.longitude']]

In [43]:
#c_cult_pd2_final

In [44]:
c_cult_pd2['@type'].values
#Necesito extraer solo la última parte de la URL

array(['https://datos.madrid.es/egob/kos/entidadesYorganismos/AuditoriosSalasConcierto',
       'https://datos.madrid.es/egob/kos/entidadesYorganismos/AuditoriosSalasConcierto',
       'https://datos.madrid.es/egob/kos/entidadesYorganismos/CentrosCulturales',
       'https://datos.madrid.es/egob/kos/entidadesYorganismos/CentrosAtencionSocial/CentrosServiciosSociales',
       'https://datos.madrid.es/egob/kos/entidadesYorganismos/CentrosCulturales',
       'https://datos.madrid.es/egob/kos/entidadesYorganismos/CentrosCulturales',
       'https://datos.madrid.es/egob/kos/entidadesYorganismos/CentrosCulturales',
       'https://datos.madrid.es/egob/kos/entidadesYorganismos/CentrosCulturales',
       'https://datos.madrid.es/egob/kos/entidadesYorganismos/AuditoriosSalasConcierto',
       'https://datos.madrid.es/egob/kos/entidadesYorganismos/CentrosCulturales',
       'https://datos.madrid.es/egob/kos/entidadesYorganismos/CentrosCulturales',
       'https://datos.madrid.es/egob/kos/entidad

In [45]:
#c_cult_pd2['Center_type']= c_cult_pd2_final['@type'].str.split('/',expand=True)[6]

In [46]:
c_cult_pd2.loc[:, 'Center_type'] = c_cult_pd2['@type'].str.split('/', expand=True)[6]
#El mensaje de advertencia que aparece es un aviso de pandas indicando 
#que estás modificando una vista de un DataFrame y no el DataFrame original.

In [47]:
c_cult_pd2

,@id,@type,id,title,relation,address.district.@id,address.area.@id,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,organization.organization-desc,organization.accesibility,organization.schedule,organization.services,organization.organization-name,Center_type
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,5989871,Auditorio Carmen Laforet (Ciudad Lineal),http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28033,CALLE JAZMIN 46,40.479608,-3.665799,Ubicado en el Centro de Mayores Carmen Laforet...,1,De 9 a 21 horas.,Aforo: 190 personas Zona wifi Cafetería,Auditorio Carmen Laforet (Ciudad Lineal),AuditoriosSalasConcierto
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,2389,Auditorio al aire libre. Parque del Calero,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28027,CALLE JOSE DEL HIERRO 3,40.436300,-3.650410,Mantenimiento: Dependiente de la Junta Munici...,1,,,Auditorio al aire libre. Parque del Calero,AuditoriosSalasConcierto
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,139451,Auditorio y sala de exposiciones Paco de Lucía...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28044,AVENIDA LAS AGUILAS 2,40.384939,-3.764196,Metro de Madrid. Línea 5. Estación: Aluche. ...,1,,Sala de exposiciones. Salón de actos y auditor...,Auditorio y sala de exposiciones Paco de Lucía...,CentrosCulturales
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,84763,Centro Comunitario Casino de la Reina,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28005,CALLE CASINO 3,40.406818,-3.704690,"Metro: Embajadores (líneas 3 y 5) Bus: M1, ...",1,"De apertura De lunes a viernes, de 8:30 a 21...",Orientación sobre recursos sociales a vecinos....,Centro Comunitario Casino de la Reina,CentrosAtencionSocial
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,1914,Centro Cultural - Centro Socio Cultural Villa ...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28042,CALLE BOTICA 7 y 10,40.473402,-3.578523,"Metro: Barajas (línea 8) Bus: 101, 105, 115",1,"De lunes a viernes, de 9 a 14 y de 15 a 17 hor...",3 salas de exposiciones Salón de actos / audit...,Centro Cultural - Centro Socio Cultural Villa ...,CentrosCulturales
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,4776440,Medialab en Matadero. Matadero Madrid,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28045,PASEO CHOPERA 14,40.392458,-3.697207,Medialab Matadero es un programa del Área de G...,1,,,Medialab en Matadero. Matadero Madrid,CentrosCulturales
114,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,10189992,Nave de Terneras del Centro Cultural Casa del ...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28045,PASEO CHOPERA 10,40.393433,-3.698578,"Metro: Legazpi (líneas 3, 6) Bus: 62, 6, 78...",1,Agosto cerrado.,Wifi Aforo: 492 personas,Nave de Terneras del Centro Cult

In [48]:
#c_cult_pd2['Center_type']= c_cult_pd2['Center_type'].replace({'CentrosCulturales':'Centros Culturales', 'AuditoriosSalasConcierto': 'Auditorios Salas Concierto', 'CentrosAtencionSocial': 'Centros Atencion Social', 'SalasExposiciones':'Salas Exposiciones'})                                  

In [49]:
c_cult_pd2.loc[:,'Center_type']= c_cult_pd2['Center_type'].replace({'CentrosCulturales':'Centros Culturales', 'AuditoriosSalasConcierto': 'Auditorios Salas Concierto', 'CentrosAtencionSocial': 'Centros Atencion Social', 'SalasExposiciones':'Salas Exposiciones'})

In [50]:
c_cult_pd2['Center_type'].unique

<bound method Series.unique of 0      Auditorios Salas Concierto
1      Auditorios Salas Concierto
2              Centros Culturales
3         Centros Atencion Social
4              Centros Culturales
                  ...            
113            Centros Culturales
114            Centros Culturales
115            Salas Exposiciones
116            Centros Culturales
117            Salas Exposiciones
Name: Center_type, Length: 118, dtype: object>

In [51]:
c_cult_pd2

,@id,@type,id,title,relation,address.district.@id,address.area.@id,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,organization.organization-desc,organization.accesibility,organization.schedule,organization.services,organization.organization-name,Center_type
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,5989871,Auditorio Carmen Laforet (Ciudad Lineal),http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28033,CALLE JAZMIN 46,40.479608,-3.665799,Ubicado en el Centro de Mayores Carmen Laforet...,1,De 9 a 21 horas.,Aforo: 190 personas Zona wifi Cafetería,Auditorio Carmen Laforet (Ciudad Lineal),Auditorios Salas Concierto
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,2389,Auditorio al aire libre. Parque del Calero,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28027,CALLE JOSE DEL HIERRO 3,40.436300,-3.650410,Mantenimiento: Dependiente de la Junta Munici...,1,,,Auditorio al aire libre. Parque del Calero,Auditorios Salas Concierto
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,139451,Auditorio y sala de exposiciones Paco de Lucía...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28044,AVENIDA LAS AGUILAS 2,40.384939,-3.764196,Metro de Madrid. Línea 5. Estación: Aluche. ...,1,,Sala de exposiciones. Salón de actos y auditor...,Auditorio y sala de exposiciones Paco de Lucía...,Centros Culturales
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,84763,Centro Comunitario Casino de la Reina,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28005,CALLE CASINO 3,40.406818,-3.704690,"Metro: Embajadores (líneas 3 y 5) Bus: M1, ...",1,"De apertura De lunes a viernes, de 8:30 a 21...",Orientación sobre recursos sociales a vecinos....,Centro Comunitario Casino de la Reina,Centros Atencion Social
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,1914,Centro Cultural - Centro Socio Cultural Villa ...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28042,CALLE BOTICA 7 y 10,40.473402,-3.578523,"Metro: Barajas (línea 8) Bus: 101, 105, 115",1,"De lunes a viernes, de 9 a 14 y de 15 a 17 hor...",3 salas de exposiciones Salón de actos / audit...,Centro Cultural - Centro Socio Cultural Villa ...,Centros Culturales
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,4776440,Medialab en Matadero. Matadero Madrid,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28045,PASEO CHOPERA 14,40.392458,-3.697207,Medialab Matadero es un programa del Área de G...,1,,,Medialab en Matadero. Matadero Madrid,Centros Culturales
114,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,10189992,Nave de Terneras del Centro Cultural Casa del ...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28045,PASEO CHOPERA 10,40.393433,-3.698578,"Metro: Legazpi (líneas 3, 6) Bus: 62, 6, 78...",1,Agosto cerrado.,Wifi Aforo: 492 personas,Nave de Terneras del Ce

In [52]:
c_cult_pd2_final= c_cult_pd2[['Center_type','title','address.street-address', 'location.latitude', 'location.longitude']]

In [53]:
c_cult_pd2_final

,Center_type,title,address.street-address,location.latitude,location.longitude
0,Auditorios Salas Concierto,Auditorio Carmen Laforet (Ciudad Lineal),CALLE JAZMIN 46,40.479608,-3.665799
1,Auditorios Salas Concierto,Auditorio al aire libre. Parque del Calero,CALLE JOSE DEL HIERRO 3,40.436300,-3.650410
2,Centros Culturales,Auditorio y sala de exposiciones Paco de Lucía...,AVENIDA LAS AGUILAS 2,40.384939,-3.764196
3,Centros Atencion Social,Centro Comunitario Casino de la Reina,CALLE CASINO 3,40.406818,-3.704690
4,Centros Culturales,Centro Cultural - Centro Socio Cultural Villa ...,CALLE BOTICA 7 y 10,40.473402,-3.578523
...,...,...,...,...,...
113,Centros Culturales,Medialab en Matadero. Matadero Madrid,PASEO CHOPERA 14,40.392458,-3.697207
114,Centros Culturales,Nave de Terneras del Centro Cultural Casa del ...,PASEO CHOPERA 10,40.393433,-3.698578
115,Salas Exposiciones,Sala de exposiciones La Lonja. Centro Cultural...,PASEO CHOPERA 10,40.393433,-3.698578
116,Centros Culturales,Salas de ensayo Pau Donés y Luis Eduardo Aute ...,CALLE ALCALDE SAINZ DE BARANDA 94,40.421218,-3.660765


In [54]:
type(c_cult_pd2_final)

pandas.core.frame.DataFrame

In [55]:
len(c_cult_pd2_final)

118

In [56]:
#c_cult_pd2_df_filter = pd.DataFrame(c_cult_pd2)
#paso a DF el json

In [57]:
#type(c_cult_pd2_df_filter)

In [58]:
#PRRRRRlat_lon= bicimad_df_filter["geometry.coordinates"].str.strip('[]').str.split(",", expand=True)

In [59]:
#PRRRRRlat_lon.columns = ['longitud','latitud']

In [60]:
#PRRRRRbicimad_df_filter = pd.concat([bicimad_df_filter, lat_lon], axis=1)
#PRRRRRbicimad_df_filter 

In [61]:
#PRRRRRbicimad_df_filter['latitud'] = bicimad_df_filter['latitud'].astype('float')
#con el método astype consigo cambiar los datosa de la columna latitud de string a float
# No me ha dejado hacer las dos columnas a la vez.

In [62]:
#PRRRRRbicimad_df_filter = bicimad_df_filter.astype({'latitud':'float', 'longitud':'float'})
#PRRRRRbicimad_df_filter

In [63]:
#PRRRRRtype(bicimad_df_filter ['latitud'][0])

In [64]:
#PRRRRRbicimad_df_filter['longitud'] = bicimad_df_filter['longitud'].astype('float')

In [65]:
#PRRRRRtype(bicimad_df_filter ['longitud'][0])

In [66]:
#PRRRRRbicimad_df_final=bicimad_df_filter[['name','address', 'latitud', 'longitud']]

In [67]:
c_cult_pd2_final= c_cult_pd2_final.rename(columns={'Center_type':'Center_type','title':'Place_of_interest', 'address.street-address':'address_center', 'location.latitude':'latitude_center', 'location.longitude':'longitude_center'})

In [68]:
c_cult_pd2_final

,Center_type,Place_of_interest,address_center,latitude_center,longitude_center
0,Auditorios Salas Concierto,Auditorio Carmen Laforet (Ciudad Lineal),CALLE JAZMIN 46,40.479608,-3.665799
1,Auditorios Salas Concierto,Auditorio al aire libre. Parque del Calero,CALLE JOSE DEL HIERRO 3,40.436300,-3.650410
2,Centros Culturales,Auditorio y sala de exposiciones Paco de Lucía...,AVENIDA LAS AGUILAS 2,40.384939,-3.764196
3,Centros Atencion Social,Centro Comunitario Casino de la Reina,CALLE CASINO 3,40.406818,-3.704690
4,Centros Culturales,Centro Cultural - Centro Socio Cultural Villa ...,CALLE BOTICA 7 y 10,40.473402,-3.578523
...,...,...,...,...,...
113,Centros Culturales,Medialab en Matadero. Matadero Madrid,PASEO CHOPERA 14,40.392458,-3.697207
114,Centros Culturales,Nave de Terneras del Centro Cultural Casa del ...,PASEO CHOPERA 10,40.393433,-3.698578
115,Salas Exposiciones,Sala de exposiciones La Lonja. Centro Cultural...,PASEO CHOPERA 10,40.393433,-3.698578
116,Centros Culturales,Salas de ensayo Pau Donés y Luis Eduardo Aute ...,CALLE ALCALDE SAINZ DE BARANDA 94,40.421218,-3.660765


In [69]:
len(c_cult_pd2_final)

118

In [70]:
c_cult_pd2_final['pointCC'] = c_cult_pd2_final.apply(lambda x:(gpd.GeoSeries([Point(x['latitude_center'], x['longitude_center'])], crs=4326).to_crs(3857)), axis=1)

In [71]:
c_cult_pd2_final

,Center_type,Place_of_interest,address_center,latitude_center,longitude_center,pointCC
0,Auditorios Salas Concierto,Auditorio Carmen Laforet (Ciudad Lineal),CALLE JAZMIN 46,40.479608,-3.665799,POINT (4506169.379 -408353.573)
1,Auditorios Salas Concierto,Auditorio al aire libre. Parque del Calero,CALLE JOSE DEL HIERRO 3,40.436300,-3.650410,POINT (4501348.277 -406636.934)
2,Centros Culturales,Auditorio y sala de exposiciones Paco de Lucía...,AVENIDA LAS AGUILAS 2,40.384939,-3.764196,POINT (4495630.879 -419330.185)
3,Centros Atencion Social,Centro Comunitario Casino de la Reina,CALLE CASINO 3,40.406818,-3.704690,POINT (4498066.408 -412691.876)
4,Centros Culturales,Centro Cultural - Centro Socio Cultural Villa ...,CALLE BOTICA 7 y 10,40.473402,-3.578523,POINT (4505478.548 -398618.638)
...,...,...,...,...,...,...
113,Centros Culturales,Medialab en Matadero. Matadero Madrid,PASEO CHOPERA 14,40.392458,-3.697207,POINT (4496467.854 -411857.100)
114,Centros Culturales,Nave de Terneras del Centro Cultural Casa del ...,PASEO CHOPERA 10,40.393433,-3.698578,POINT (4496576.366 -412010.047)
115,Salas Exposiciones,Sala de exposiciones La Lonja. Centro Cultural...,PASEO CHOPERA 10,40.393433,-3.698578,POINT (4496576.366 -412010.047)
116,Centros Culturales,Salas de ensayo Pau Donés y Luis Eduardo Aute ...,CALLE ALCALDE SAINZ DE BARANDA 94,40.421218,-3.660765,POINT (4499669.387 -407792.080)


In [72]:
dataset_combined= c_cult_pd2_final.merge(bicimad_df_final, how='cross')
#Con axix=1 indicamos que queremos unirlo por filas, si ponemos axis=0 se uniría por columnas.
#Decido hacerlo por filas porque el nº de entradas es menor

In [73]:
dataset_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31152 entries, 0 to 31151
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Center_type        31152 non-null  object  
 1   Place_of_interest  31152 non-null  object  
 2   address_center     31152 non-null  object  
 3   latitude_center    31152 non-null  float64 
 4   longitude_center   31152 non-null  float64 
 5   pointCC            31152 non-null  geometry
 6   nameBM             31152 non-null  object  
 7   addressBM          31152 non-null  object  
 8   latitudeBM         31152 non-null  float64 
 9   longitudeBM        31152 non-null  float64 
 10  pointBM            31152 non-null  geometry
dtypes: float64(4), geometry(2), object(5)
memory usage: 2.6+ MB


In [74]:
dataset_combined.head(15)

,Center_type,Place_of_interest,address_center,latitude_center,longitude_center,pointCC,nameBM,addressBM,latitudeBM,longitudeBM,pointBM
0,Auditorios Salas Concierto,Auditorio Carmen Laforet (Ciudad Lineal),CALLE JAZMIN 46,40.479608,-3.665799,POINT (4506169.379 -408353.573),1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,POINT (4499223.648 -412373.285)
1,Auditorios Salas Concierto,Auditorio Carmen Laforet (Ciudad Lineal),CALLE JAZMIN 46,40.479608,-3.665799,POINT (4506169.379 -408353.573),1b - Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603,POINT (4499234.670 -412347.499)
2,Auditorios Salas Concierto,Auditorio Carmen Laforet (Ciudad Lineal),CALLE JAZMIN 46,40.479608,-3.665799,POINT (4506169.379 -408353.573),2 - Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842,POINT (4499599.341 -412820.321)
3,Auditorios Salas Concierto,Auditorio Carmen Laforet (Ciudad Lineal),CALLE JAZMIN 46,40.479608,-3.665799,POINT (4506169.379 -408353.573),3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917,POINT (4500679.707 -412940.307)
4,Auditorios Salas Concierto,Auditorio Carmen Laforet (Ciudad Lineal),CALLE JAZMIN 46,40.479608,-3.665799,POINT (4506169.379 -408353.573),4 - Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587,POINT (4500485.867 -412457.329)
5,Auditorios Salas Concierto,Auditorio Carmen Laforet (Ciudad Lineal),CALLE JAZMIN 46,40.479608,-3.665799,POINT (4506169.379 -408353.573),5 - Fuencarral,Calle Fuencarral nº 108,40.428520,-3.702050,POINT (4500482.260 -412397.369)
6,Auditorios Salas Concierto,Auditorio Carmen Laforet (Ciudad Lineal),CALLE JAZMIN 46,40.479608,-3.665799,POINT (4506169.379 -408353.573),6 - Colegio Arquitectos,Calle Hortaleza nº 63,40.424148,-3.698447,POINT (4499995.571 -411995.447)
7,Auditorios Salas Concierto,Auditorio Carmen Laforet (Ciudad Lineal),CALLE JAZMIN 46,40.479608,-3.665799,POINT (4506169.379 -408353.573),7 - Hortaleza,Calle Hortaleza nº 75,40.425191,-3.697771,POINT (4500111.633 -411920.094)
8,Auditorios Salas Concierto,Auditorio Carmen Laforet (Ciudad Lineal),CALLE JAZMIN 46,40.479608,-3.665799,POINT (4506169.379 -408353.573),8 - Alonso Martínez,Plaza de Alonso Martínez nº 5,40.427868,-3.695440,POINT (4500409.702 -411660.045)
9,Auditorios Salas Concierto,Auditorio Carmen Laforet (Ciudad Lineal),CALLE JAZMIN 46,40.479608,-3.665799,POINT (4506169.379 -408353.573),9 - Plaza de San Miguel,Plaza de San Miguel nº 9,40.415606,-3.709508,POINT (4499044.647 -413229.374)


In [75]:
from shapely.geometry import Point
import geopandas as gpd   # conda install -c conda-forge geopandas

def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)


In [76]:
dataset_combined['distance'] = dataset_combined.apply(lambda row: distance_meters(row['latitude_center'], row['longitude_center'], row['latitudeBM'], row['longitudeBM']), axis=1)

In [77]:
#dataset_combined['distance'] = dataset_combined.apply(lambda x: gpd.GeoSeries([Point(x['latitude_center'], x['longitude_center'])], crs=4326).to_crs(3857).distance(gpd.GeoSeries([Point(x['latitudeBM'], x['longitudeBM'])], crs=4326).to_crs(3857))[0], axis=1)


In [78]:
dataset_combined

,Center_type,Place_of_interest,address_center,latitude_center,longitude_center,pointCC,nameBM,addressBM,latitudeBM,longitudeBM,pointBM,distance
0,Auditorios Salas Concierto,Auditorio Carmen Laforet (Ciudad Lineal),CALLE JAZMIN 46,40.479608,-3.665799,POINT (4506169.379 -408353.573),1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,POINT (4499223.648 -412373.285),8025.039900
1,Auditorios Salas Concierto,Auditorio Carmen Laforet (Ciudad Lineal),CALLE JAZMIN 46,40.479608,-3.665799,POINT (4506169.379 -408353.573),1b - Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603,POINT (4499234.670 -412347.499),8002.601695
2,Auditorios Salas Concierto,Auditorio Carmen Laforet (Ciudad Lineal),CALLE JAZMIN 46,40.479608,-3.665799,POINT (4506169.379 -408353.573),2 - Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842,POINT (4499599.341 -412820.321),7944.636258
3,Auditorios Salas Concierto,Auditorio Carmen Laforet (Ciudad Lineal),CALLE JAZMIN 46,40.479608,-3.665799,POINT (4506169.379 -408353.573),3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917,POINT (4500679.707 -412940.307),7153.644319
4,Auditorios Salas Concierto,Auditorio Carmen Laforet (Ciudad Lineal),CALLE JAZMIN 46,40.479608,-3.665799,POINT (4506169.379 -408353.573),4 - Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587,POINT (4500485.867 -412457.329),7010.215935
...,...,...,...,...,...,...,...,...,...,...,...,...
31147,Salas Exposiciones,Salón y sala de exposiciones en Junta Municipa...,AVENIDA RAFAELA YBARRA 41,40.382387,-3.710862,POINT (4495346.729 -413380.339),257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,40.438960,-3.729970,POINT (4501644.435 -415511.958),6648.676997
31148,Salas Exposiciones,Salón y sala de exposiciones en Junta Municipa...,AVENIDA RAFAELA YBARRA 41,40.382387,-3.710862,POINT (4495346.729 -413380.339),258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,40.443750,-3.726990,POINT (4502177.656 -415179.522),7063.895630
31149,Salas Exposiciones,Salón y sala de exposiciones en Junta Municipa...,AVENIDA RAFAELA YBARRA 41,40.382387,-3.710862,POINT (4495346.729 -413380.339),259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,40.443420,-3.726930,POINT (4502140.920 -415172.829),7026.667534
31150,Salas Exposiciones,Salón y sala de exposiciones en Junta Municipa...,AVENIDA RAFAELA YBARRA 41,40.382387,-3.710862,POINT (4495346.729 -413380.339),260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,40.448332,-3.727295,POINT (4502687.744 -415213.491),7566.435542


In [79]:
dataset_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31152 entries, 0 to 31151
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Center_type        31152 non-null  object  
 1   Place_of_interest  31152 non-null  object  
 2   address_center     31152 non-null  object  
 3   latitude_center    31152 non-null  float64 
 4   longitude_center   31152 non-null  float64 
 5   pointCC            31152 non-null  geometry
 6   nameBM             31152 non-null  object  
 7   addressBM          31152 non-null  object  
 8   latitudeBM         31152 non-null  float64 
 9   longitudeBM        31152 non-null  float64 
 10  pointBM            31152 non-null  geometry
 11  distance           31152 non-null  float64 
dtypes: float64(5), geometry(2), object(5)
memory usage: 2.9+ MB


In [80]:
newdataset_combined = dataset_combined.sort_values(by=['distance'], axis=0, ascending=True)

In [81]:
%time
newdataset_combined2 = newdataset_combined.loc[newdataset_combined.groupby('Place_of_interest')['distance'].idxmin()][['Place_of_interest','Center_type','address_center','nameBM','addressBM','distance',]].reset_index(drop=True)


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


In [82]:
newdataset_combined2

,Place_of_interest,Center_type,address_center,nameBM,addressBM,distance
0,Auditorio Carmen Laforet (Ciudad Lineal),Auditorios Salas Concierto,CALLE JAZMIN 46,249 - Manuel Caldeiro,Paseo de la Castellana nº 298,2177.692829
1,Auditorio al aire libre. Parque del Calero,Auditorios Salas Concierto,CALLE JOSE DEL HIERRO 3,229 - C.D. Concepción,Calle Jose del Hierro nº 5,247.636031
2,Auditorio y sala de exposiciones Paco de Lucía...,Centros Culturales,AVENIDA LAS AGUILAS 2,224 - Caramuel,Calle Caramuel nº 51,4804.535443
3,Centro Comunitario Casino de la Reina,Centros Atencion Social,CALLE CASINO 3,48 - Embajadores 2,Glorieta de Embajadores nº 2,302.658663
4,Centro Cultural - Centro Socio Cultural Villa ...,Centros Culturales,CALLE BOTICA 7 y 10,215 - Gutierre de Cetina,Calle Gutierre de Cetina nº 77,8497.910663
...,...,...,...,...,...,...
113,Medialab en Matadero. Matadero Madrid,Centros Culturales,PASEO CHOPERA 14,165 - Entrada Matadero,Paseo de la Chopera nº 14,62.272540
114,Nave de Terneras del Centro Cultural Casa del ...,Centros Culturales,PASEO CHOPERA 10,165 - Entrada Matadero,Paseo de la Chopera nº 14,127.977223
115,Sala de exposiciones La Lonja. Centro Cultural...,Salas Exposiciones,PASEO CHOPERA 10,165 - Entrada Matadero,Paseo de la Chopera nº 14,127.977223
116,Salas de ensayo Pau Donés y Luis Eduardo Aute ...,Centros Culturales,CALLE ALCALDE SAINZ DE BARANDA 94,185 - Quinta Fuente del Berro,Calle Sancho Dávila esquina Paseo Marqués de Z...,438.623971


In [ ]:
type(newdataset_combined2)

In [ ]:
''''parser = argparse.ArgumentParser(description='1 for all places and 2 for one place')
parser.add_argument('-1', action='store_const', const=1, help='Export all places')
parser.add_argument('-2', action='store', dest='place_of_interest', help='Export one place')
args = parser.parse_args()

if args.place_of_interest:
    eleccion("2")
else:
    eleccion("1")'''

In [83]:
##opción A
def eleccion(tabla):
    if tabla == 1:
        csv_allplaces = newdataset_combined2.to_csv('table_allplaces.csv')
        print("your complete table is generating")
    elif tabla == 2:
        place_of_user = input("Write your place of interest: ")
        place_for_user = newdataset_combined2[newdataset_combined2['Place_of_interest'].str.contains(place_of_user, case=False)]
        if not place_for_user.empty:
            place_for_user.to_csv('places_for_user.csv')
            print("your specific table is generating")
        else:
            print("The place is not in the list")
    else:
        print("the message is not correct")

parser = argparse.ArgumentParser(description='Specific places or all places')
parser.add_argument("myplace", type=str, help= 'Two options: "places" or "myplace"')
parser.add_argument("places", type=str,dest='place_of_interest', help= 'Two options: "places" or "myplace"')
args = parser.parse_args()

if args.places == "myplace":
    eleccion(tabla=2)
elif args.places == "places":
    eleccion(tabla=1)
else:
    print("Invalid option")


ValueError: dest supplied twice for positional argument

In [ ]:
##OPCION B

def eleccion(tabla, place_of_interest=None):
    if tabla == 1:
        csv_allplaces = newdataset_combined2.to_csv('table_allplaces.csv')
        print("your complete table is generating")
    elif tabla == 2:
        if place_of_interest is None:
            place_of_interest = input("Write your place of interest: ")
        place_for_user = newdataset_combined2[newdataset_combined2['Place_of_interest'].str.contains(place_of_interest, case=False)]
        if not place_for_user.empty:
            place_for_user.to_csv('places_for_user.csv')
            print("your specific table is generating")
        else:
            print("The place is not in the list")
    else:
        print("the message is not correct")
parser = argparse.ArgumentParser(description='Specific places or all places')
parser.add_argument("myplace", type=str, help= 'Two options: "places" or "myplace"')
parser.add_argument("places", type=str,dest='place_of_interest', help= 'Two options: "places" or "myplace"')
args = parser.parse_args()

if args.places == "myplace":
    eleccion(tabla=2)
elif args.places == "places":
    eleccion(tabla=1)
else:
    print("Invalid option")

In [ ]:
Auditorio Carmen Laforet (Ciudad Lineal)

In [ ]:
parser = argparse.ArgumentParser(description='Specific places or all places')
parser.add_argument("--myplace", type=str, help= 'Two options: "places" or "myplace"')
parser.add_argument("--places", type=str, help= 'Two options: "places" or "myplace"')
args = parser.parse_args()
def eleccion(args):
    if args.places:
        csv_allplaces = newdataset_combined2.to_csv('table_allplaces.csv')
        print("Your complete table is generating")
    elif args.myplace:
        place_of_interest = args.myplace
        place_for_user = newdataset_combined2[newdataset_combined2['Place_of_interest'].str.contains(place_of_interest, case=False)]
        if not place_for_user.empty:
            place_for_user.to_csv('places_for_user.csv')
            print("Your specific table is generating")
        else:
            print("The place is not in the list")
    else:
        print("The message is not correct")
        



